In [122]:
import networkx as nx
import spacy
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as spatial
import re
import json
from nltk.corpus import stopwords

In [123]:
stop = stopwords.words('english')
stop.append("'s")
with open('mj.json', 'r') as f:
    data = json.load(f)

H = nx.read_gexf("graph.gexf")
pos = nx.spring_layout(H)
val = list(pos.values())
x = []
y = []
for i in val:
    x.append(i[0])
    y.append(i[1])
# plt.scatter(x,y)
# plt.show()

In [124]:
#UTILITY FUNCTIONS

def get_pos_indexes(lst):

    index = []

    #Iterate over the list using indexes
    for i in range(len(lst)-1):

        #If first element was positive, add 0 as index
        if i == 0:
            if lst[i] > 0:
                index.append(0)
        #If successive values are negative and positive, i.e indexes switch over, collect the positive index
        if lst[i] < 0 and lst[i+1] > 0:
            index.append(i+1)

    #If index list was empty, all negative characters were encountered, hence add -1 to index
    if len(index) == 0:
        index = [-1]

    return index

def preprocessor(text):
    if type(text)!=str:
        text = str(text)
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(text)
    lemma_list = []
    for token in doc:
        if token.is_alpha is True and token.is_stop is False and token not in stop:
            lemma_list.append(token.lemma_)
    return_list=[]
    for i in range(len(lemma_list)):
        temp_text = re.sub('<[^>]*>', '', lemma_list[i])
        temp_text = re.sub('[\W]+', '', temp_text.lower())
        return_list.append(temp_text)
    return(return_list)

def evaluate_keywords(proposed,groundtruth):
    
    proposed_set = set(proposed)
    true_set = set(groundtruth)
    
    true_positives = len(proposed_set.intersection(true_set))
    
    if len(proposed_set)==0:
        precision = 0
    
    else:
        precision = true_positives/float(len(proposed)) 
    
    if len(true_set)==0:
        recall = 0
    
    else:
        recall = true_positives/float(len(true_set))
        
    if precision + recall > 0:
        f1 = 2*precision*recall/float(precision + recall)
    
    else:
        f1 = 0
    
    return (precision, recall, f1)

In [125]:
#Calculate Radius which covers 10 percent of total nodes from center

meh = []
for i in range(0, 100, 5):
    meh.append(len(point_tree.query_ball_point([0.0, 0.0], i/100))- len(H.nodes())/10)
index = get_pos_indexes(meh)
r = (index[0]*5)/100

In [126]:
#Find nodes which are in this Radius

coordinate = np.column_stack((x, y))
point_tree = spatial.cKDTree(coordinate)

index = point_tree.query_ball_point([0.0, 0.0], r)
wordlist= list(pos.keys())

#Candidate Keywords
candidate_key = []
for i in index:
    candidate_key.append(wordlist[i].split('.',1)[0])

#Ground Truth
final_res = data[0]['keywords'].split('\n')

In [127]:
evaluate_keywords(candidate_key, final_res)

(0.15384615384615385, 0.4782608695652174, 0.23280423280423282)